

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/CLASSIFICATION_De_SENTIMENT.ipynb)






# **Sentiment Analysis of German texts**

## 1. Colab Setup

In [ ]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.8

In [2]:
import json
import pandas as pd
import numpy as np

import sparknlp
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

## 2. Start Spark Session

In [3]:
spark = sparknlp.start()
print ("Spark NLP Version :", sparknlp.version())
spark

Spark NLP Version : 4.2.8


## 3. Some sample examples

In [4]:
text_list = [
"""Spiel und Meisterschaft nicht spannend genug? Muss man jetzt den Videoschiedsrichter kontrollieren? Ich bin entsetzt,dachte der darf nur bei krassen Fehlentscheidungen ran. So macht der Fussball keinen Spass mehr.""",             
"""Habe gestern am Mittwoch den werder Podcast vermisst. Wie schnell man sich an etwas gewöhnt und darauf freut. Danke an Plainsman für die guten Interviews und den Einblick hinter die Kulissen von werderbremen. Angenehme Winterpause weiterhin!""",
"""ich mag den Schiri nicht, denn der ist vergiftet!""",
"""Die Szenen folgen ruckartig aufeinander, die Dialoge sind theatralisch, die schauspielerischen Leistungen kommen nicht wirklich über den Film hinaus. Nur die Musik von Vivaldi rettet den Tag. Eine große Enttäuschung.""",
"""ManCity Guardiola und seine Gang, ihr seid Arschlöcher. Ich habe gerade ein Vermögen durch deine Schuld deine Bayern dort verloren""",
]

## 4. Define Spark NLP pipeline

In [5]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

embeddings = BertSentenceEmbeddings\
    .pretrained('labse', 'xx') \
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

sentimentClassifier = ClassifierDLModel.pretrained("classifierdl_bert_sentiment", "de") \
  .setInputCols(["sentence_embeddings"]) \
  .setOutputCol("class_")   
     
nlpPipeline = Pipeline(
    stages=[
      document, 
      embeddings,
      sentimentClassifier
      ])

labse download started this may take some time.
Approximate size to download 1.7 GB
[OK!]
classifierdl_bert_sentiment download started this may take some time.
Approximate size to download 22.2 MB
[OK!]


## 5. Run the pipeline

In [6]:
df = spark.createDataFrame(text_list, StringType()).toDF("text")
result = nlpPipeline.fit(df).transform(df)

## 6. Visualize results

In [7]:
result.select(F.explode(F.arrays_zip(result.document.result, 
                                     result.class_.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("document"),
              F.expr("cols['1']").alias("class")).show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|document                                                                                                                                                                                                                                         |class   |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Spiel und Meisterschaft nicht spannend genug? Muss man jetzt den Videoschiedsrichter kontrollieren? Ich bin entsetzt,dachte der darf nur bei krassen Fehlentscheidungen ran. So macht der Fussball keinen Spass mehr.                           